## HW4: Neural Network

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import scipy.io as sio
from scipy import optimize

In [4]:
data = sio.loadmat('data/ex3weights.mat')
Theta1 = data['Theta1']
Theta2 = data['Theta2']

data = sio.loadmat('data/ex3data1.mat')
X = data['X']
Y = data['y']

k=10
Y_ = np.zeros((5000,k))
for i in xrange(1,k):
    Y_[i*500:(i+1)*500,i-1] = 1.
Y_[0:500,-1] = 1.

print X.shape, Y.shape
print Theta1.shape, Theta2.shape

(5000, 400) (5000, 1)
(25, 401) (10, 26)


In [5]:
# Here is a class for creating NNs:
class NN(object):
    """
    A neural network with arbitrary layers and neurans in each layer.
    
    Arguments:
    x  -  x must be an array, containing number of neurans in eaxh layer.
    """
    def __init__(self, x):
        self.num_layer = np.shape(x)
        self.theta1 = np.zeros((x[1],x[0]))
        self.theta2 = np.zeros((x[2],x[1]))
        self.act1 = np.zeros(x[0])
        self.act2 = np.zeros(x[1])
        self.act3 = np.zeros(x[2])
        self.act1[0] = 1  ## this is input layer
        self.act2[0] = 1  ## this is hidden layer
        
    def __call__(self):
        pass
    
    def assignWeigth(self,theta1,theta2):
        self.theta1 = theta1
        self.theta2 = theta2
        return 
    
    def assignInput(self,x):
        self.theta1 = theta1
        self.theta2 = theta2
        return

    def forwardProp(self,theta,input_): 
        return np.dot(theta,input_)

In [6]:
info = [401,26,10]
myNN = NN(info)
# print myNN.theta1.shape
myNN.assignWeigth(Theta1,Theta2)
print myNN.theta1[:,:].shape
print myNN.theta2[:,:].shape

theta_all = np.append(myNN.theta1.flatten(),myNN.theta2.flatten())

(25, 401)
(10, 26)


In [7]:
sigmoid = lambda x: 1./(1+np.exp(-x))
sigmoid_prime = lambda x: sigmoid(x)*(1.-sigmoid(x))

In [8]:
def forwardProp(input_,weight):
    """
    One step forward propagation.
    
    Arguments:
    weight
    input_
    """    
    z = weight[:,0] + np.dot(weight[:,1:],input_)
    a = sigmoid(z)
    return z,a

In [11]:
def calc_accuracy (theta_all, X):
    
    theta1 = theta_all[0:myNN.theta1.size].reshape(myNN.theta1.shape)
    theta2 = theta_all[myNN.theta1.size: ].reshape(myNN.theta2.shape)

    wrong_tot = 0
    a3 = np.zeros((5000,k))
    for i in xrange(5000):
        dummy,a2 = forwardProp(X[i,:],theta1)
        dummy,a3[i,:] = forwardProp(a2,theta2)
    for i in xrange(0,5000):
        index = np.floor(i/500)
        c = np.argmax(a3[i,:])
        if int(index) is 0:
            if (int(c) is not 9): ## or k-1
                wrong_tot = wrong_tot + 1
        else:
            if (int(c) is not int(index)-1):
    #             print i,c,index
                wrong_tot = wrong_tot + 1
    print "Accuracy is", 1-float(wrong_tot)/5000
    
    return

In [12]:
calc_accuracy (theta_all, X)

Accuracy is 0.9752


In [13]:
def cost_function_nn_regularized(theta_all, X, Y_, lambda_):
    """
    Arguments:
    
    Y       -  The observed output (1D array)
    lambda_ -  The regularization parameter
    """
    num_observations = Y.shape[0]
    theta1 = theta_all[0:myNN.theta1.size].reshape(myNN.theta1.shape)
    theta2 = theta_all[myNN.theta1.size: ].reshape(myNN.theta2.shape)
    
    a3 = np.zeros((5000,k))
    for i in xrange(5000):
        dummy,a2 = forwardProp(X[i,:],theta1)
        dummy,a3[i,:] = forwardProp(a2,theta2)
    
#     print "a3 in cost = ", a3[0,:]
    
    return 1./num_observations * np.sum(-(Y_*np.log(a3))-((1.-Y_)*np.log(1.-a3))) + \
                                 lambda_/(2.*num_observations) * (np.sum(theta1[:,1:]**2.) + np.sum(theta2[:,1:]**2.))  

In [14]:
def gradient_nn (theta_all, X, Y_, lambda_):
    
    num_observations = Y.shape[0]

    delta3 = np.zeros((5000,10))
    delta2 = np.zeros((5000,25))
    Delta1 = np.zeros(myNN.theta1.shape)
    Delta2 = np.zeros(myNN.theta2.shape)
    
    a2 = np.zeros((5000,25))
    a3 = np.zeros((5000,10))  ## or k
    z2 = np.zeros((5000,25))
    z3 = np.zeros((5000,10))
    
    a1_ = np.concatenate((np.ones((5000,1)),X),axis=1)
    
    theta1 = theta_all[0:myNN.theta1.size].reshape(myNN.theta1.shape)
    theta2 = theta_all[myNN.theta1.size: ].reshape(myNN.theta2.shape)
    
    for i in xrange(num_observations):
        z2[i,:],a2[i,:] = forwardProp(X[i,:],theta1)
        z3[i,:],a3[i,:] = forwardProp(a2[i,:],theta2)
        a2_ = np.concatenate((np.ones((5000,1)),a2),axis=1)
        delta3[i,:] = a3[i,:]-Y_[i,:]
        delta2[i,:] = np.dot(theta2[:,1:].T,delta3[i,:])*sigmoid_prime(z2[i,:])
        Delta1 = Delta1 + np.dot(delta2[i,:][:,None],a1_[i,:][:,None].T)
        Delta2 = Delta2 + np.dot(delta3[i,:][:,None],a2_[i,:][:,None].T)
        
    Delta1 = Delta1/num_observations
    Delta2 = Delta2/num_observations
    
    Delta1[:,1:] = Delta1[:,1:] + lambda_/num_observations * Delta1[:,1:]
    Delta2[:,1:] = Delta2[:,1:] + lambda_/num_observations * Delta2[:,1:]
        
#     print "a3 in gradient = ", a3[0,:]
    
    return np.append(Delta1.flatten(),Delta2.flatten())

In [15]:
theta_all = np.append(myNN.theta1.flatten(),myNN.theta2.flatten())
theta_all = np.random.uniform(-0.12,0.12,myNN.theta1.size+myNN.theta2.size)
# print X
cost_function_nn_regularized(theta_all, X, Y_, 0.)
gradient_nn(theta_all,X,Y_,0.)

array([-0.01933458,  0.        ,  0.        , ...,  0.19358526,
        0.22540307,  0.16708017])

In [17]:
def myOptimize3(num_iter,lambda_):
    """
    This is using fmin_cg from python, much faster
    """ 
    x0 = np.random.uniform(-0.12,0.12,myNN.theta1.size+myNN.theta2.size)
#     print x0
    theta1 = x0[0:myNN.theta1.size].reshape(myNN.theta1.shape)
    theta2 = x0[myNN.theta1.size: ].reshape(myNN.theta2.shape)
#     a3 = np.zeros((5000,k))
#     for i in xrange(5000):
#         dummy,a2 = forwardProp(X[i,:],theta1)
#         dummy,a3[i,:] = forwardProp(a2,theta2)
        
#     print "a3 in optimize = ", a3[0,:]

#     print cost_function_nn_regularized(x0, a3, Y_, 0.)
#     print gradient_nn(x0, a3, Y_, 0.)
    
    result = optimize.fmin_cg(cost_function_nn_regularized, x0=x0, fprime=gradient_nn, \
                              args=(X, Y_, lambda_), maxiter=num_iter, disp=False,\
                              full_output=True)
#     print result[1]
#     print gradient_nn(result[0], a3, Y_, 0.)
    return result[0],result[1]

In [29]:
a,b = myOptimize3(50,0.)

In [32]:
print a,b
calc_accuracy(a,X)

[-0.52183642 -0.01214162  0.04045423 ...,  1.58558673 -0.30238964
  3.65893099] 0.23577168374
Accuracy is 0.9702
